In [1]:
import pandas as pd 
import string
import regex as re
from pyarabic import araby

In [7]:
#### Import tweets with sentiment analysis 
data=pd.read_csv('Results/predictions_tweet.csv',header=0, delimiter=";",  encoding="utf-8-sig") 
data.head() 

,tweet,prediction_sentiment
0,تصويري عبدالرحمن الدغيلبي عيش السعودية قطفنا ا...,1.0
1,تصويري عبدالرحمن الدغيلبي عيش السعودية قطفنا ا...,1.0
2,تصويري متنوع الارشيف عيش السعودية عبدالرحمن ال...,1.0
3,عيش السعودية وطني قلبي ينبع أملج السعودية,1.0
4,بوابة الوفد الدنيا لتنشيط السياحة الداخلية,1.0


In [8]:
data.tweet.count()

209657

In [9]:
data.groupby('prediction_sentiment').count().tweet

prediction_sentiment
0.0     22407
1.0    140569
2.0     46681
Name: tweet, dtype: int64

In [10]:
#### Import keywords used to search location within the tweet text
pd_places =pd.read_csv('Data/listeplaces2.csv',header=0, delimiter=";",  encoding="utf-8-sig") 
pd_places.head(5)

,place
0,الرياض
1,المجمعة
2,المجمعه
3,القويعية
4,القويعيه


In [11]:
places= pd_places['place'].tolist() 
### Eliminate espaces in places list
list_place=[]
for i in range(len(places)):
	x=places[i].strip()
	list_place.append(x)

In [12]:
#### Normalized_places is a list that will be used on normalization of location name
normalized_places=pd.read_csv('Data/normalized_places.csv',header=0, delimiter=";",  encoding="utf-8-sig") 
normalized_places.head()

,place,normalized_place
0,ابها,أبها
1,الطايف,الطائف
2,جده,جدة
3,بريده,بريدة
4,املج,أملج


In [13]:
list_place1=[]
list_place2=[]
for i in range(normalized_places.shape[0]):
	x1=normalized_places.place[i].strip()
	x2=normalized_places.normalized_place[i].strip()
	list_place1.append(x1)
	list_place2.append(x2)


In [14]:
normalized_places['place']=list_place1
normalized_places['normalized_place']=list_place2

In [15]:
normalized_places=normalized_places.drop_duplicates()

In [16]:

def location(text):
	location_i= " " 
	list_w=str(text).split()
	for i in range(len(list_w)) :
		if list_w[i]=='وادي' and i<len(list_w)-1 and len(list_w[i+1])!=0   :
			location_i +='وادي ' + list_w[i+1]
			location_i += " "
		elif list_w[i]=='وادي'  and i<len(list_w)-2  and len(list_w[i+2])!=0 and (list_w[i+1]=='بني' or list_w[i+1]=='بن' ) :
			location_i +='وادي ' + list_w[i+1] + list_w[i+2]
			location_i += " "
		elif list_w[i]=='قرية' and i<len(list_w)-1   and len(list_w[i+1])!=0 :
			location_i +='قرية ' + list_w[i+1]
			location_i += " "
		elif list_w[i]=='جزر' and i<len(list_w)-1 and len(list_w[i+1])!=0   :
			location_i +='جزر ' + list_w[i+1]
			location_i += " "
		elif list_w[i]=='بدر' and i<len(list_w)-1 and list_w[i+1]=='الجنوب':
			location_i +='  بدر الجنوب ' 
		elif list_w[i]=='عيون' and i<len(list_w)-1 and list_w[i+1]=='الجواء':
			location_i +=' عيون الجواء '
		elif list_w[i]=='رياض'and i<len(list_w)-1 and list_w[i+1]=='الخبراء':
			location_i +=' رياض الخبراء   '
		elif list_w[i]=='مهد' and i<len(list_w)-1 and list_w[i+1]=='الذهب':
			location_i +=' مهد الذهب '
		elif list_w[i]=='بني' and i<len(list_w)-1 and list_w[i+1]=='تميم':
			location_i +='  بني تميم '
		elif list_w[i]=='بني' and i<len(list_w)-1 and list_w[i+1]=='حسن':
			location_i +='  بني حسن '
		elif list_w[i]=='الحدود' and i<len(list_w)-1 and (list_w[i+1]=='الشماليه' or list_w[i+1]=='الشمالية'):
			location_i +=' الحدود الشمالية   '
		else:
			for place in list_place: 
				if list_w[i]==place:
					location_i += place
					location_i += " "
	x=list(dict.fromkeys(location_i.split()))
	location=' '.join([str(elem) for elem in x])
	return location


In [17]:
list_eliminated=['بني','الشمالية','وادي','قرية']

In [23]:
tweet="عيش السعوديه وادي حنيفه"
location(tweet)

'وادي حنيفه'

In [27]:
tweet="تعتبر السياحة السعودية القطاعات الناشئة تعد السياحة الدينية أهم ركائزها كونها مهد الدين الإسلامي يجعلها محل جذب سياحي"
print('the location is:',location(tweet))

the location is: 


In [18]:
def normalize_location(text):
	location_i= " "
	list_w=str(text).split() # to split text into words
	for i in range(len(list_w)): # for each word in the text 
		x_i=""	
		for j in range(normalized_places.shape[0]):
			if list_w[i].strip()==normalized_places.place[j].strip(): # if the word is equal to a place noun 
                                                                      #  from the normalized list
				x_i +=normalized_places.normalized_place[j] # the normalized from of this noum is returned to the x_i list

			else:
				x_i +=""
		if len(x_i)==0:  # if the length(x_i) is 0 ( x_i don't contain any place) the initial noun of the place is returned 
			location_i += list_w[i]
			location_i += " "
		else: 
			location_i += x_i # else the location will be equal the normalized form
			location_i += " "

	x=list(dict.fromkeys(location_i.split())) # eliminate the duplicate places noun , a list is created
	location=' '.join([str(elem) for elem in x]) # join  the created list below into a string
	if location.strip() in list_eliminated: # if the location contain only one the word in the list_eliminated 
                                            # list_eliminated=['بني','الشمالية','وادي','قرية']
                                            # the location will be set as empty 
		location='' 
	else: 
		location
	return location



In [19]:
## Apply the location function 
location_list=data['tweet'].apply(lambda x: location(x))
data['location_0']=location_list

In [20]:
## Apply the normalize_location function 
location_normalized_list=data['location_0'].apply(lambda x: normalize_location(x))
data['location']=location_normalized_list

In [16]:
# Top n visted places (n=50)
n=50

most_visited_places_list=data.groupby('location').count().tweet.nlargest(n).index.values
most_visited_places=most_visited_places_list[1:n-1]
column_names = ["location", "Negative", "Neutral","Positive" ]
data_result= pd.DataFrame(columns = column_names)

In [17]:

for i in range(len(most_visited_places)-1):
	xi=data[data['location']==most_visited_places[i]]
	data_result.loc[i,'location']=most_visited_places[i]
	data_result.loc[i,'Negative']=len(xi[xi['prediction_sentiment']==0.0])
	data_result.loc[i,'Neutral']=len(xi[xi['prediction_sentiment']==1.0])
	data_result.loc[i,'Positive']=len(xi[xi['prediction_sentiment']==2.0])

In [18]:
list_of_visted_places=data.groupby('location').count().tweet.sort_values(ascending=False)

In [19]:
list_of_visted_places.head()

location
          136977
الرياض      5640
العلا       4269
حائل        3514
الطائف      3389
Name: tweet, dtype: int64

In [20]:
list_of_visted_places.to_csv(r'Output/list_of_visted_places.csv', index = True, header=True, sep=";", encoding='utf-8')

In [21]:
data.to_csv(r'Output/data_tweets_with_location.csv', index = False, header=True, sep=";", encoding='utf-8')

In [22]:
data_result.to_csv(r'Results/most_visited_places.csv', index = False, header=True, sep=";", encoding='utf-8')